In [2]:
!pip install autoxgb plotly catboost stats impyute
!pip install pmdarima

In [10]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer
from pathlib import Path

In [18]:
comp_dir = Path('../input/store-sales-time-series-forecasting')
train = pd.read_csv(comp_dir / 'train.csv')
test = pd.read_csv(comp_dir / 'test.csv')

In [19]:
train.head()

In [21]:
df_data = train.copy()

In [22]:
df_data.shape

In [ ]:
for column in df_data.columns:
  print(column + " : " + str(df_data[column].nunique()))
print("---------")


In [23]:
df_data.isnull().sum()

In [ ]:
# check for stationarity
from statsmodels.tsa.stattools import adfuller
import numpy as np
def check_stationarity(ts):
    dftest = adfuller(ts)
    adf = dftest[0]
    pvalue = dftest[1]
    critical_value = dftest[4]['5%']
    if (pvalue < 0.05) and (adf < critical_value):
        print('The series is stationary')
    else:
        print('The series is NOT stationary')

In [24]:
df_data['store_nbr_family'] = train['store_nbr'].astype(str) + train['family'].astype(str)
df_data.head()

In [25]:
store_nbr_family = df_data.store_nbr_family.unique()
print(len(store_nbr_family))

In [26]:
df_data.isnull().sum()

In [41]:
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA
for eachStoreFamily in store_nbr_family:
  eachStoreFamilyDf = df_data[df_data["store_nbr_family"] == eachStoreFamily]
  eachStoreFamilyDf['date'] = pd.DatetimeIndex(eachStoreFamilyDf['date'])
  eachStoreFamilyDf = eachStoreFamilyDf[["date", "sales"]]
  eachStoreFamilyDf = eachStoreFamilyDf.set_index('date')
  eachTs = eachStoreFamilyDf['sales']
    
    
  ###To be deleted
  '''stepwise_model = pm.auto_arima(eachTs,start_p=1, d=1, start_q=1,
                            max_p=12, max_q=12,
                            m=0,
                            seasonal=False, stationary=True,
                            information_criterion='aic',
                            stepwise=False,
                            suppress_warnings=True)
  stepwise_model.fit(eachTs)'''
  ###############

  model = ARIMA(eachTs, order=(1,0,3))
  model_fit = model.fit()
  store_nbr_family_predictions[eachStoreFamily] = model_fit.forecast(16)
  print(stepwise_model)
  #break

In [55]:
storeNbrFamilyCount = dict()
for eachStoreFamily in store_nbr_family:
  storeNbrFamilyCount[eachStoreFamily] = 0

In [56]:
tempDict = dict()
for key,value in store_nbr_family_predictions.items():
    tempDict[key] = value.reset_index(drop=True)
    

In [57]:
test['sales'] = 0

for i, row in test.iterrows():
    storeNbrFamily = str(row['store_nbr']) + str(row['family'])
    test.at[i,'sales'] = tempDict[storeNbrFamily][storeNbrFamilyCount[storeNbrFamily]]
    storeNbrFamilyCount[storeNbrFamily] = storeNbrFamilyCount[storeNbrFamily] + 1

In [58]:
test.to_csv('submission_arima.csv', index=False)